In [4]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements, solve_ritz
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


In [2]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_8/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_8/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 21)
(21, 21)
rl errV: 2.6368292973414723e-05
abs errV: 7.428768900793086e-05
errt: 0.00034576375364847226
errh: 4.908986218433534e-05
errht: 0.00013382891090997002


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H8_Hamiltonian.mtx').tocsr()
H_correct_8 = generate_Hamiltonian_with_occupation_number(H_correct.real, 8)
e_test, v_test = sparse.linalg.eigsh(H_correct_8, which = 'SA', k = 10)
e_ground = e_test[0]
e_1st_ex = e_test[1]

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [5]:
g_phy =  eri.transpose(0, 2, 1, 3)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)


#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

[1, 16, 62, 100, 154, 100, 62, 16, 1]
<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
#initial1 = generate_single_state(14, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
initial = generate_single_state(8, [3,3,3,3,0,0,0,0])
# initial = ptn.mps.add_mps(initial1, initial2)
# initial.orthonormalize('left')
# initial.orthonormalize('right')


We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [8]:
N_Krylov_1 = 25
psi_original = copy.deepcopy(initial)
max_bond_Krylov = 150
trunc_tol = 1e-11
foldername_1 = f"/work_fast/ge49cag/code_datas/H8_Krylov"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
#generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername_1)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [9]:
#N_use = 30
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
#H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [12]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov, e_ground, mpo_ref)

(3.1056646837181887+0j)
(1.4171435471643337+0j)
(0.5881029804806595+0j)
(0.13968977241255232+0j)
(0.0007551916160366545+0j)


In [ ]:
# (2.372950682823987+0j)
# (1.3619017579585355+0j)
# (0.6261388289291965+0j)
# (0.2514166942954521+0j)
# (0.07860499329798287+0j)
# (0.03354498244057602+0j)

Restart:

In [11]:
N_Krylov_2 = 35
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 150
trunc_tol = 1e-11
foldername_2= f"/work_fast/ge49cag/code_datas/H8_Krylov_1ex_energy_continue"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)


[1, 2, 4, 8, 16, 32, 64, 124, 153, 124, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 127, 150, 127, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
13
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
14
[1, 2, 4, 8, 16, 32, 64, 128, 150, 127, 64, 32, 16, 8, 4, 2, 1]
15
[1, 2, 4, 8, 16, 32, 64, 128, 150, 128, 64, 32, 16, 8, 4, 2, 1]
16
[1,

In [12]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:212: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/thc_experiments/../pytenet/global_krylov_method.py:111: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [13]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.014881602303610464+0j)
(0.0018350796427935023+0j)
(6.0683247237847127e-05+0j)
(1.7247480208482102e-06+0j)
(2.2193896676014901e-07+0j)
(1.5797739472134253e-07+0j)
(1.5300436828624697e-07+0j)


In [68]:
max_index = np.argmax((v_ritz_2.as_vector()))
#v_ritz_2.as_vector()[max_index]
print(bin(max_index))

0b1001100110100110
